### 重叠聚类图

In [ ]:
# =====================================================
# 国家共识聚类空间分布地图绘制（修正版）
# =====================================================
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("🗺️ 国家共识聚类空间分布地图绘制")
print("=" * 70)

# ==================== 配置区 ====================
class Config:
    MAPPING_FILE = r"F:\Desktop\CAPMF\国家共识聚类映射.csv"
    OUTPUT_DIR = r"F:\Desktop\CAPMF\聚类空间分布地图"
    
    # 地图样式配置
    FIGSIZE = (20, 12)
    DPI = 300
    
    # 配色方案（Nature期刊风格，支持最多20个簇）
    CLUSTER_COLORS = [
        '#E64B35', '#4DBBD5', '#00A087', '#3C5488', '#F39B7F',
        '#8491B4', '#91D1C2', '#DC0000', '#7E6148', '#B09C85',
        '#E377C2', '#7F7F7F', '#BCBD22', '#17BECF', '#AEC7E8',
        '#FFBB78', '#98DF8A', '#FF9896', '#C5B0D5', '#C49C94'
    ]
    
    # 无聚类国家的颜色
    NO_CLUSTER_COLOR = '#EEEEEE'
    BORDER_COLOR = '#333333'
    BORDER_WIDTH = 0.5
    
    # 中文字体
    FONT_MAIN = "Microsoft YaHei"
    FONT_SIZE_TITLE = 16
    FONT_SIZE_LEGEND = 11
    FONT_SIZE_ANNOTATION = 9

# ==================== 数据加载 ====================
print("\n📂 **步骤1：加载数据**")
config = Config()

# 读取映射表
df_mapping = pd.read_csv(config.MAPPING_FILE, encoding='utf-8-sig')
print(f"✅ 映射表加载完成：{len(df_mapping)} 条记录")
print(f"   K值范围：{sorted(df_mapping['K值'].unique())}")
print(f"   涉及国家：{df_mapping['国家'].nunique()} 个")

# 加载世界地图数据（使用在线URL）
print("\n📥 正在下载世界地图数据...")
try:
    url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
    world = gpd.read_file(url)
    print(f"✅ 世界地图加载完成：{len(world)} 个国家/地区")
except Exception as e:
    print(f"❌ 地图加载失败：{e}")
    print("💡 请检查网络连接或安装 geopandas: pip install geopandas")
    raise

# ==================== 国家名称标准化映射 ====================
# 处理中英文国家名称差异
country_name_mapping = {
    # 中文 -> 英文（geopandas使用英文名）
    '丹麦': 'Denmark',
    '保加利亚': 'Bulgaria',
    '加拿大': 'Canada',
    '匈牙利': 'Hungary',
    '卢森堡': 'Luxembourg',
    '南非': 'South Africa',
    '奥地利': 'Austria',
    '希腊': 'Greece',
    '德国': 'Germany',
    '意大利': 'Italy',
    '拉脱维亚': 'Latvia',
    '挪威': 'Norway',
    '捷克': 'Czechia',
    '斯洛伐克': 'Slovakia',
    '斯洛文尼亚': 'Slovenia',
    '法国': 'France',
    '波兰': 'Poland',
    '爱尔兰': 'Ireland',
    '爱沙尼亚': 'Estonia',
    '瑞典': 'Sweden',
    '瑞士': 'Switzerland',
    '立陶宛': 'Lithuania',
    '罗马尼亚': 'Romania',
    '芬兰': 'Finland',
    '英国': 'United Kingdom',
    '荷兰': 'Netherlands',
    '葡萄牙': 'Portugal',
    '西班牙': 'Spain',
    '克罗地亚': 'Croatia',
    '比利时': 'Belgium',
    '塞浦路斯': 'Cyprus',
    '马耳他': 'Malta',
    '中国': 'China',
    '日本': 'Japan',
    '韩国': 'South Korea',
    '印度': 'India',
    '印度尼西亚': 'Indonesia',
    '泰国': 'Thailand',
    '越南': 'Vietnam',
    '马来西亚': 'Malaysia',
    '菲律宾': 'Philippines',
    '新加坡': 'Singapore',
    '澳大利亚': 'Australia',
    '新西兰': 'New Zealand',
    '美国': 'United States of America',
    '墨西哥': 'Mexico',
    '巴西': 'Brazil',
    '阿根廷': 'Argentina',
    '智利': 'Chile',
    '哥伦比亚': 'Colombia',
    '秘鲁': 'Peru',
    '俄罗斯': 'Russia',
    '乌克兰': 'Ukraine',
    '土耳其': 'Turkey',
    '埃及': 'Egypt',
    '沙特阿拉伯': 'Saudi Arabia',
    '阿联酋': 'United Arab Emirates',
    '以色列': 'Israel',
    '伊朗': 'Iran',
    '伊拉克': 'Iraq',
    '约旦': 'Jordan',
    '黎巴嫩': 'Lebanon',
    '科威特': 'Kuwait',
    '卡塔尔': 'Qatar',
    '巴林': 'Bahrain',
    '阿曼': 'Oman',
    '也门': 'Yemen',
    '叙利亚': 'Syria',
    '巴基斯坦': 'Pakistan',
    '孟加拉国': 'Bangladesh',
    '斯里兰卡': 'Sri Lanka',
    '尼日利亚': 'Nigeria',
    '肯尼亚': 'Kenya',
    '埃塞俄比亚': 'Ethiopia',
    '坦桑尼亚': 'Tanzania',
    '乌干达': 'Uganda',
    '加纳': 'Ghana',
    '摩洛哥': 'Morocco',
    '阿尔及利亚': 'Algeria',
    '突尼斯': 'Tunisia',
    '利比亚': 'Libya',
    '津巴布韦': 'Zimbabwe',
    '博茨瓦纳': 'Botswana',
    '纳米比亚': 'Namibia',
    '莫桑比克': 'Mozambique',
    '安哥拉': 'Angola',
    '赞比亚': 'Zambia',
    '马拉维': 'Malawi',
}

# 标准化映射表中的国家名
df_mapping['国家_英文'] = df_mapping['国家'].map(country_name_mapping)

# 检查未匹配的国家
unmatched = df_mapping[df_mapping['国家_英文'].isna()]['国家'].unique()
if len(unmatched) > 0:
    print(f"\n⚠️ 以下国家未找到英文映射，请手动添加到 country_name_mapping：")
    for country in unmatched:
        print(f"   '{country}': '',")
    # 使用原名称作为备选
    df_mapping['国家_英文'].fillna(df_mapping['国家'], inplace=True)

# ==================== 绘图函数 ====================
def plot_cluster_map(k_value, df_data, world_gdf, config):
    """绘制单个K值的聚类空间分布地图"""
    
    print(f"\n{'='*60}")
    print(f"🎨 绘制 K={k_value} 的聚类空间分布地图")
    print(f"{'='*60}")
    
    # 筛选当前K值的数据
    df_k = df_data[df_data['K值'] == k_value].copy()
    clusters = sorted(df_k['共识聚类ID'].unique())
    
    print(f"  📊 簇数量：{len(clusters)}")
    for cid in clusters:
        countries = df_k[df_k['共识聚类ID'] == cid]['国家'].tolist()
        print(f"     簇{cid}: {len(countries)}国 - {', '.join(countries[:5])}{'...' if len(countries)>5 else ''}")
    
    # 创建国家到簇的映射
    country_to_cluster = dict(zip(df_k['国家_英文'], df_k['共识聚类ID']))
    
    # 为地图数据添加簇ID列
    world_copy = world_gdf.copy()
    world_copy['cluster_id'] = world_copy['NAME'].map(country_to_cluster)
    
    # 设置配色
    color_map = {}
    for i, cid in enumerate(clusters):
        color_map[cid] = config.CLUSTER_COLORS[i % len(config.CLUSTER_COLORS)]
    
    world_copy['color'] = world_copy['cluster_id'].apply(
        lambda x: color_map.get(x, config.NO_CLUSTER_COLOR) if pd.notna(x) else config.NO_CLUSTER_COLOR
    )
    
    # 创建图表
    fig, ax = plt.subplots(1, 1, figsize=config.FIGSIZE)
    
    # 绘制地图
    world_copy.plot(
        ax=ax,
        color=world_copy['color'],
        edgecolor=config.BORDER_COLOR,
        linewidth=config.BORDER_WIDTH
    )
    
    # 设置标题
    ax.set_title(
        f'国家共识聚类空间分布（K={k_value}）',
        fontsize=config.FONT_SIZE_TITLE,
        fontweight='bold',
        pad=20,
        fontname=config.FONT_MAIN
    )
    
    # 去除坐标轴
    ax.set_axis_off()
    
    # 添加图例
    legend_elements = []
    for cid in clusters:
        countries_in_cluster = df_k[df_k['共识聚类ID'] == cid]['国家'].tolist()
        label = f'簇{cid} ({len(countries_in_cluster)}国)'
        legend_elements.append(
            mpatches.Patch(
                facecolor=color_map[cid],
                edgecolor=config.BORDER_COLOR,
                label=label
            )
        )
    
    # 添加"其他国家"图例
    legend_elements.append(
        mpatches.Patch(
            facecolor=config.NO_CLUSTER_COLOR,
            edgecolor=config.BORDER_COLOR,
            label='其他国家'
        )
    )
    
    ax.legend(
        handles=legend_elements,
        loc='lower left',
        fontsize=config.FONT_SIZE_LEGEND,
        frameon=True,
        framealpha=0.9,
        edgecolor='black',
        prop={'family': config.FONT_MAIN}
    )
    
    # 统计信息文本框
    stats_text = (
        f"簇数量: {len(clusters)}\n"
        f"涉及国家: {len(df_k)}个\n"
        f"总国家数: {len(world_gdf)}"
    )
    ax.text(
        0.02, 0.98,
        stats_text,
        transform=ax.transAxes,
        fontsize=config.FONT_SIZE_ANNOTATION,
        verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
        fontname=config.FONT_MAIN
    )
    
    plt.tight_layout()
    
    # 保存图片
    output_dir = Path(config.OUTPUT_DIR)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_file = output_dir / f'K{k_value}_聚类空间分布地图.png'
    plt.savefig(
        output_file,
        dpi=config.DPI,
        bbox_inches='tight',
        facecolor='white'
    )
    plt.close()
    
    print(f"  ✅ 地图已保存：{output_file.name}")
    
    return str(output_file)

# ==================== 批量绘制 ====================
print("\n" + "="*70)
print("🚀 **开始批量绘制聚类空间分布地图**")
print("="*70)

k_values = sorted(df_mapping['K值'].unique())
output_files = []

for k in k_values:
    try:
        output_file = plot_cluster_map(
            k_value=k,
            df_data=df_mapping,
            world_gdf=world,
            config=config
        )
        output_files.append(output_file)
    except Exception as e:
        print(f"  ❌ K={k} 绘制失败：{e}")
        import traceback
        traceback.print_exc()

# ==================== 生成汇总报告 ====================
print("\n" + "="*70)
print("📋 **生成汇总信息**")
print("="*70)

summary = []
for k in k_values:
    df_k = df_mapping[df_mapping['K值'] == k]
    clusters = df_k['共识聚类ID'].unique()
    summary.append({
        'K值': k,
        '簇数量': len(clusters),
        '涉及国家数': len(df_k),
        '簇ID列表': ', '.join(map(str, sorted(clusters)))
    })

df_summary = pd.DataFrame(summary)
summary_file = Path(config.OUTPUT_DIR) / '聚类空间分布汇总.csv'
df_summary.to_csv(summary_file, index=False, encoding='utf-8-sig')

print(f"✅ 汇总信息已保存：{summary_file}")
print("\n📊 汇总统计：")
print(df_summary.to_string(index=False))

# ==================== 完成总结 ====================
print("\n" + "="*70)
print("✅ **所有聚类空间分布地图绘制完成！**")
print("="*70)
print(f"📁 **输出目录**：{config.OUTPUT_DIR}")
print(f"🗺️ **生成地图**：{len(output_files)} 张")
print(f"📋 **K值范围**：{min(k_values)} - {max(k_values)}")
print("\n💡 **输出文件列表**：")
for i, file in enumerate(output_files, 1):
    print(f"   {i}. {Path(file).name}")
